In [ ]:
# Importing the required libraries
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import matplotlib.pyplot as plt
import pytorch_lightning as pl

# Setting the random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Defining the batch size, available GPUs, and number of workers
BATCH_SIZE=1
AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS=int(os.cpu_count() / 2)

This block of code imports various libraries required for building a deep learning model using PyTorch. It also sets the random seed to 42 for reproducibility of results.

Furthermore, it defines the batch size for the data loader, the number of available GPUs, and the number of workers for the data loader to use.
***

In [ ]:
# Define CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.wavelet_files = os.listdir(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.wavelet_files)

    def __getitem__(self, idx):
        file_name=self.wavelet_files[idx]
        wavelet_path = os.path.join(self.root_dir, file_name)
        wavelet = pd.read_csv(wavelet_path,header=None).values
        wavelet_tensor = torch.tensor(wavelet, dtype=torch.float32)
        wavelet_tensor = wavelet_tensor.view(6, -1, wavelet_tensor.shape[1])
        Label_Tag=file_name.split('_')
        label=[]
        for n in [1,3,5,7,9,11,13]:
            temp_num=float(Label_Tag[n])
            label.append(temp_num)
        label=torch.tensor(label)
    
        return wavelet_tensor, label

This block of code defines a custom dataset class called ```CustomDataset```. The class takes in two parameters: ```root_dir``` and ```transform```, which represent the directory where the data is stored and any data transformation to be applied respectively.

The ```__init__``` method initializes the dataset object with the specified ```root_dir``` and ```transform```. The ```__len__``` method returns the length of the dataset, which is the number of files in the ```root_dir```.

The ```__getitem__``` method retrieves the data at the given index ```idx```. It reads the file name from the list of wavelet files in the ```root_dir```. It then reads the wavelet data from the CSV file at the path ```wavelet_path``` using pandas and converts it to a PyTorch tensor. The tensor is then reshaped to have 6 channels, with each channel having a shape that depends on the original shape of the wavelet data.

The method then extracts the label information from the file name and converts it to a PyTorch tensor. It returns both the wavelet tensor and its corresponding label tensor as a tuple.
***

In [ ]:
# Reading wavelet image from CSV file
test = pd.read_csv(\
    'Data/Wavelet_Image/Ron_50_Roff_50_Pulse_10_Vds_200_Vgson_17.5_Vgsoff_3_Resamplefac_50_id_1.csv',\
          header=None)

# Printing the size of the wavelet image
print(test.size)

This block of code reads a wavelet image from a CSV file and stores it in a pandas DataFrame object called ```test```. The CSV file is located in the directory ```Data/Wavelet_Image``` and has the name ```Ron_50_Roff_50_Pulse_10_Vds_200_Vgson_17.5_Vgsoff_3_Resamplefac_50_id_1.csv```.

The ```header=None``` argument indicates that there are no column headers in the CSV file.

The second line of code prints the size of the ```test``` DataFrame using the ```size``` attribute. This prints the total number of elements in the DataFrame, which is equivalent to the number of rows multiplied by the number of columns.
***

In [ ]:
## Converting the 2D data matrix to a 3D matrix

class CustomDataModule(pl.LightningDataModule):
    def __init__(self, csv_root, transform=None, batch_size=32, num_workers=0):
        super().__init__()
        self.csv_root = csv_root   # Directory path where the CSV file is stored
        self.batch_size = batch_size   # The batch size for the data loader
        self.num_workers = num_workers   # The number of worker processes for loading the data
        self.transform = transform   # Optional data transformation to be applied

    def prepare_data(self):
        pass   # Placeholder for any data preparation step, if needed

    def setup(self, stage=None):
        self.dataset = CustomDataset(self.csv_root, transform=self.transform)   # Initialize the CustomDataset class with the specified CSV directory and transform
        #self.dataset_train, self.dataset_val = random_split(self.dataset,[int(len(self.dataset)*0.7),len(self.dataset)-int(len(self.dataset)*0.7)])
        self.dataset_train = self.dataset   # Set the training dataset to be the entire dataset

    def train_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)   # Return a DataLoader object for the training data, which shuffles the data and divides it into batches

    def val_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)   # Return a DataLoader object for the validation data, which is set to be the same as the training data

    def test_dataloader(self):
        return None   # No test data is used for this model, so return None


This block of code defines a PyTorch Lightning data module called ```CustomDataModule``` for loading the wavelet data. The ```CustomDataModule``` class inherits from the ```pl.LightningDataModule``` class.

The ```__init__``` method initializes the data module object with the specified CSV root directory, batch size, and number of workers. The ```transform``` parameter is an optional argument that represents any data transformations to be applied.

The ```prepare_data``` method does not have any functionality in this case, so it is left blank.

The ```setup``` method initializes the dataset object with the specified CSV root directory and transform, using the ```CustomDataset``` class defined earlier.

The ```train_dataloader``` method returns a PyTorch DataLoader object for training the model, using the DataLoader class from PyTorch. This ```DataLoader``` loads the training data from the ```dataset_train``` object, shuffles the data, and divides it into batches of size ```batch_size```.

The ```val_dataloader``` method returns a PyTorch DataLoader object for validation, which is set to be the same as the training data.

The ```test_dataloader``` method returns ```None```, indicating that no test data is used for this model.
***

In [ ]:
# Setting the root directory of the dataset
root_dir = 'Data/Wavelet_Image/'

# Defining the transformations for the dataset using PyTorch's Compose function
transformations = transforms.Compose([
    transforms.ToTensor(),
])

# Creating a custom dataset and dataloader using the CustomDataModule class
data_module = CustomDataModule(root_dir, transform=transformations)

This block of code sets the root directory for the dataset to ```'Data/Wavelet_Image/'```.

Next, data transformations are defined using PyTorch's ```Compose``` function. The ```transforms.ToTensor()``` function is used to convert the wavelet data to PyTorch tensors.

Finally, a custom dataset and dataloader are created using the ```CustomDataModule``` class defined earlier. The ```root_dir``` and ```transform``` parameters are passed to the ```CustomDataModule``` constructor to specify the directory of the CSV files and the data transformation to be applied. The resulting ```data_module``` object can be used to load the wavelet data for training and validation.
***

In [ ]:
# Set up the data module and data loader
data_module.setup()

# Retrieve the training data from the data loader
train_dataloader = data_module.train_dataloader()

# Get the first batch of data from the data loader
i, l = next(iter(train_dataloader))

This block of code sets up the data module and data loader for the wavelet data using the ```setup``` method of the ```data_module``` object.

Next, the training data is retrieved from the data loader using the ```train_dataloader``` method of the ```data_module``` object.

Finally, the first batch of data from the training data loader is obtained using the ```next``` function and unpacked into two variables: ```i``` (input data) and ```l``` (labels). The ```next``` function returns the next batch of data as a tuple, which is why it is unpacked into two separate variables.
***

In [ ]:
# Printing the shape of the input data and labels
print(i.shape)
print(l.shape)

- Input Size [batch, 7]

- Wavelet Size [batch, 6, 87, 25000]

- We need to change the code while considering the data size.

This block of code prints the shape of the input data and labels obtained from the first batch of the training data loader.

The ```i.shape``` prints the shape of the input data i, which should be a 3D tensor with dimensions ```(batch_size, num_channels, height, width)```, where ```batch_size``` is the number of samples in the batch, ```num_channels``` is the number of channels in the data (in this case, 6), and ```height``` and ```width``` are the height and width of the wavelet image, respectively.

The ```l.shape``` prints the shape of the labels ```l```, which should be a 2D tensor with dimensions ```(batch_size, num_labels)```, where ```batch_size``` is the number of samples in the batch and ```num_labels``` is the number of labels in the data (in this case, 7).
***

### Discriminator



In [ ]:
# A discriminator model that determines if an image is real or fake, outputting a single value between 0 and 1

## TODO: Change the Channel and input size

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        # Simple CNN architecture
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 1)
  
    def forward(self, x):
        # Apply convolutional and max pooling layers with ReLU activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # Flatten the tensor so it can be fed into the fully connected layers
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        # Apply sigmoid activation to output a value between 0 and 1
        return torch.sigmoid(x)

This block of code defines a PyTorch module called ```Discriminator``` that takes in wavelet images as input and outputs a single value between 0 and 1 indicating whether the input image is real or fake.

The ```nn.Module``` class is used as the parent class for this custom module.

The ```__init__``` method initializes the convolutional and fully connected layers of the model. The convolutional layers apply filters to the input wavelet images and the max pooling layers reduce the spatial dimensions of the output. The fully connected layers are used to reduce the output of the convolutional layers to a single value.

The ```forward``` method applies the convolutional and max pooling layers with ReLU activation to the input tensor ```x```. The output is then flattened and passed through the fully connected layers with ReLU activation and dropout regularization. Finally, the output is passed through a sigmoid activation function to produce a single value between 0 and 1.

The ```TODO``` comment indicates that the channel and input size should be changed to fit the specific dataset being used.
***

### Generator

In [ ]:
## A generator model that takes a latent space vector as input and outputs a wavelet image

## TODO: Change the Channel and input size

class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        # Define the layers of the generator
        self.lin1 = nn.Linear(latent_dim, 7*7*64)  # [n, 256, 7, 7]
        self.ct1 = nn.ConvTranspose2d(64, 32, 4, stride=2) # [n, 64, 16, 16]
        self.ct2 = nn.ConvTranspose2d(32, 16, 4, stride=2) # [n, 16, 34, 34]
        self.conv = nn.Conv2d(16, 1, kernel_size=7)  # [n, 1, 28, 28]
    

    def forward(self, x):
        # Pass the input through a linear layer and reshape
        x = self.lin1(x)
        x = F.relu(x)
        x = x.view(-1, 64, 7, 7)  #256
        
        # Upsample to 16x16 (64 feature maps)
        x = self.ct1(x)
        x = F.relu(x)
        
        # Upsample to 34x34 (16 feature maps)
        x = self.ct2(x)
        x = F.relu(x)
        
        # Apply a convolutional layer to produce a 28x28 wavelet image with a single channel
        return self.conv(x)

This block of code defines a PyTorch module called ```Generator``` that takes a latent space vector as input and generates a wavelet image as output.

The ```nn.Module``` class is used as the parent class for this custom module.

The ```__init__``` method initializes the layers of the generator. The linear layer is used to transform the input latent space vector to a 3D tensor. The convolutional transpose layers are used to upsample the tensor, increasing the spatial dimensions and decreasing the number of channels. The final convolutional layer produces a 28x28 wavelet image with a single channel.

The ```forward``` method takes the input tensor ```x``` and passes it through the layers of the generator, applying ReLU activation after each layer. The final output is a wavelet image of size 28x28 with a single channel.

The ```TODO``` comment indicates that the channel and input size should be changed to fit the specific dataset being used.
***

### GAN

In [ ]:
## TODO: Change the input Data using the label

class GAN(pl.LightningDataModule):
    def __init__(self, latent_dim=100, lr=0.0002):
        super().__init__()
        # Save the hyperparameters and initialize the generator and discriminator
        self.save_hyperparameters()
        self.generator = Generator(latent_dim=self.hparams.latent_dim)
        self.discriminator = Discriminator()
        
        # Create validation noise vector
        self.validation_z = torch.randn(6, self.hparams.latent_dim)
        
    def forward(self, z):
        return self.generator(z)
    
    def adverarial_loss(self, y_hat,y):
        # Calculate binary cross-entropy loss
        return F.binary_cross_entropy(y_hat, y)
    
    def training_step(self, batch, batch_dim, optimizer_idx):   
        real_imgs, labels = batch
        
        # Sample noise
        z = torch.randn(real_imgs.shape[0], self.hparams.latent_dim)
        z = z.type_as(real_imgs)
        
        if optimizer_idx == 0:
            # Train the generator: maximize log(D(G(z)))
            fake_imgs = self(z)
            y_hat = self.discriminator(fake_imgs)
            
            y = torch.ones(real_imgs.size(0), 1)
            y = y.type_as(real_imgs)
            
            g_loss = self.adverarial_loss(y_hat, y)
            
            log_dict = {"g_loss" : g_loss }
            return {"loss": g_loss, "progress bar" : log_dict, "log": log_dict}
        
        if optimizer_idx == 1:
            # Train the discriminator: maximize log(D(x)) + log(1 - D(G(z)))
            y_hat_real = self.discriminator(real_imgs)
            y_real = torch.ones(real_imgs.size(0), 1)
            y_real = y_real.type_as(real_imgs)
            real_loss = self.adverarial_loss(y_hat_real, y_real)
            
            y_hat_fake = self.discriminator(self(z))
            y_fake = torch.zeros(real_imgs.size(0), 1)
            y_fake = y_fake.type_as(real_imgs)
            fake_loss = self.adverarial_loss(y_hat_fake, y_fake)
            
            d_loss = (real_loss + fake_loss) / 2
            
            log_dict = {"d_loss" : d_loss }
            return {"loss": d_loss, "progress bar" : log_dict, "log": log_dict}
                
    def configure_optimizers(self):
        lr=self.hparams.lr
        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr)
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr)
        return [opt_g, opt_d], []
    
    def plot_imgs(self):
        # Generate and plot validation images
        z = self.validation_z.type_as(self.generator.lin1.weight)
        sample_imgs = self(z).cpu()
        
        print('epoch', self.current_epoch)
        fig = plt.figure()
        for i in range(sample_imgs.size(0)):
            plt.subplot(2,3,i+1)
            plt.tight_layout()
            plt.imshow(sample_imgs.detach()[i,0,:,:],cmap='gray_r',interpolation='none')
            plt.title("Generated Data")
            plt.xticks([])
            plt.yticks([])
            plt.axes('off')
        plt.show()
    
    def on_epoch_end(self):
        # Call plot_imgs() at the end of each epoch
        self.plot_imgs()

This code block defines a PyTorch Lightning module called ```GAN``` that implements a generative adversarial network (GAN).

The ```__init__``` method initializes the generator and discriminator networks, and creates a noise vector for validation.

The ```forward``` method of the ```GAN``` class passes the noise vector through the generator network to generate images.

The ```adverarial_loss``` method calculates the binary cross-entropy loss.

The ```training_step``` method trains the generator and discriminator using the binary cross-entropy loss. It first trains the generator and then the discriminator.

The ```configure_optimizers``` method sets the optimizers for the generator and discriminator.

The ```plot_imgs``` method generates validation images and plots them.

The ```on_epoch_end``` method is called at the end of each epoch and it calls the ```plot_imgs``` method.
***

In [ ]:
# Set up the MNIST data module
dm = MNISTDataModule()

# Create an instance of the GAN model
model = GAN()

This code block creates an instance of the ```MNISTDataModule``` class and assigns it to the variable ```dm```. It also creates an instance of the ```GAN``` class and assigns it to the variable ```model```.
***

In [ ]:
model.plot_imgs()

```model.plot_imgs()``` generates validation images using the generator network and plots them. This method is defined in the ```GAN``` class and uses the validation noise vector ```validation_z``` defined in the constructor to generate the images. It then plots the generated images using Matplotlib.
***

In [ ]:
# Set up the trainer object
trainer = pl.Trainer(max_epochs=20, gpus=AVAIL_GPUS)

# Train the model
trainer.fit(model, dm)

This code block sets up a ```Trainer``` object with a maximum number of epochs and the number of available GPUs. It then trains the ```model``` using the ```Trainer``` object and the ```dm``` data module.

The ```fit``` method of the ```Trainer``` object trains the ```model``` for the specified number of epochs using the data provided by the data module. During training, the ```on_epoch_end``` method of the ```GAN``` model is called after each epoch, which generates and plots validation images.

After training is complete, the ```Trainer``` object returns a ```TrainerResult``` object with information about the training process, such as the final training and validation losses.
***